In [1]:
import sys
quic_path = '/home/quic/QUIC-Projects'
if not quic_path in sys.path:
    sys.path.append(quic_path)

In [2]:
import numpy as np 
from matplotlib import pyplot as plt

In [3]:
# for SVM
from classifiers.convex.svm import BinarySVM
from classifiers.kernel import Kernel
from classifiers.datasets.dataloader import Example_4x2
# for QASVM
from classifiers.quantum import Qasvm_Mapping_4x2
from classifiers.quantum.qasvm import QASVM
from classifiers.optimizer import SPSA
from qiskit.aqua import QuantumInstance
from qiskit.ignis.mitigation import CompleteMeasFitter
from qiskit.circuit.library import RealAmplitudes, EfficientSU2, NLocal, TwoLocal
from qiskit.providers.aer import QasmSimulator
from qiskit import IBMQ
# for logging & visualization
import logging
from classifiers.callback import SimplePMStorage, SimpleStorage, ParamsPlatoStopping
from torch.utils.tensorboard import SummaryWriter
from classifiers.visualization import Plot_Data, Plot_Log_From_SPSA, Plot_SVM, compare_svm_and_qasvm

writer = SummaryWriter()
storage = SimpleStorage(writer)

if IBMQ.active_account() is None:
    IBMQ.load_account()
provider = IBMQ.get_provider(hub='ibm-q-kaist', group='internal', project='default')
backend = provider.get_backend('ibmq_toronto')
backend

<IBMQBackend('ibmq_toronto') from IBMQ(hub='ibm-q-kaist', group='internal', project='default')>

In [4]:
X, y = Example_4x2(True)()
var_form = RealAmplitudes(2, reps=2)
quantum_instance = QuantumInstance(backend=QasmSimulator.from_backend(backend), shots=2**13, basis_gates=backend.configuration().basis_gates, measurement_error_mitigation_cls=CompleteMeasFitter, optimization_level=3)
qasvm = QASVM(X, y, var_form=var_form, quantum_instance=quantum_instance, C=1, k=0.1).primal

Device reported a gate error parameter greater than maximum allowed value (1.000000 > 0.800000). Truncating to maximum value.
Device model returned a depolarizing error parameter greater than maximum allowed value (1.066957 > 1.066667). Truncating to maximum value.
Device reported a gate error parameter greater than maximum allowed value (1.000000 > 0.800000). Truncating to maximum value.
Device model returned a depolarizing error parameter greater than maximum allowed value (1.066984 > 1.066667). Truncating to maximum value.
Device reported a gate error parameter greater than maximum allowed value (1.000000 > 0.800000). Truncating to maximum value.
Device model returned a depolarizing error parameter greater than maximum allowed value (1.066958 > 1.066667). Truncating to maximum value.
Device reported a gate error parameter greater than maximum allowed value (1.000000 > 0.800000). Truncating to maximum value.
Device model returned a depolarizing error parameter greater than maximum al

Device reported a gate error parameter greater than maximum allowed value (1.000000 > 0.800000). Truncating to maximum value.
Device model returned a depolarizing error parameter greater than maximum allowed value (1.067076 > 1.066667). Truncating to maximum value.
Device reported a gate error parameter greater than maximum allowed value (1.000000 > 0.800000). Truncating to maximum value.
Device model returned a depolarizing error parameter greater than maximum allowed value (1.067774 > 1.066667). Truncating to maximum value.
Device reported a gate error parameter greater than maximum allowed value (1.000000 > 0.800000). Truncating to maximum value.
Device model returned a depolarizing error parameter greater than maximum allowed value (1.067903 > 1.066667). Truncating to maximum value.
Device reported a gate error parameter greater than maximum allowed value (1.000000 > 0.800000). Truncating to maximum value.
Device model returned a depolarizing error parameter greater than maximum al

In [5]:
qasvm.quantum_instance = QuantumInstance(backend=QasmSimulator())

In [10]:
from tqdm.notebook import tqdm
epochs = 2**10
last_avg = epochs/2**3
hyperparams = {'c0': 0.27161860782919073, 'c1': 0.1, 'c2': 0.602, 'c3': 0.101, 'c4': 0}
optimizer = SPSA(qasvm.cost_fn, qasvm.parameters, **hyperparams)
early_stopping = ParamsPlatoStopping(patiance=10, tol=1)

for epoch in tqdm(range(epochs)):
    optimizer.step()
    print(qasvm.parameters)
    storage(epoch, qasvm.cost_fn, qasvm.parameters)
    if early_stopping(np.array(list(optimizer.params.values())), epoch):
        break


  0%|          | 0/1024 [00:00<?, ?it/s]

{Parameter(θ[0]): -1.2746144120060623, Parameter(θ[1]): -2.4815789390379073, Parameter(θ[2]): 0.9934316648680708, Parameter(θ[3]): -2.664554403537928, Parameter(θ[4]): -0.1162801546035307, Parameter(θ[5]): 0.2908966426850853}


NotImplementedError: Got <class 'qiskit.circuit.parameter.Parameter'>, but numpy array, torch tensor, or caffe2 blob name are expected.

In [7]:
list(zip(optimizer.params.values(), list(optimizer.params.keys())))

[(-1.874085167566581, Parameter(θ[0])),
 (-3.081049694598426, Parameter(θ[1])),
 (0.3939609093075521, Parameter(θ[2])),
 (-2.065083647977409, Parameter(θ[3])),
 (-0.7157509101640493, Parameter(θ[4])),
 (-0.3085741128754333, Parameter(θ[5]))]

In [8]:
help(dict.__new__)

Help on built-in function __new__:

__new__(*args, **kwargs) method of builtins.type instance
    Create and return a new object.  See help(type) for accurate signature.



In [9]:
len(qasvm.parameters)

6